# LEZIONE 10 - Gradient Boosting: Ensemble Sequenziale

---

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. Comprendere l'intuizione del boosting e come differisce dal bagging
2. Distinguere Random Forest da Gradient Boosting in termini di strategia
3. Analizzare il trade-off bias-variance nei modelli ad alberi
4. Riconoscere l'overfitting nel boosting e come mitigarlo
5. Padroneggiare i parametri chiave: n_estimators, learning_rate, max_depth
6. Confrontare empiricamente Random Forest vs Gradient Boosting

---

## Prerequisiti

- Lezione 9: Decision Tree e Random Forest
- Lezione 8: Validazione, Overfitting e Generalizzazione
- Concetti di bias e varianza
- numpy, pandas, matplotlib, scikit-learn

---

## Indice

1. SEZIONE 1 - Teoria
2. SEZIONE 2 - Mappa Mentale
3. SEZIONE 3 - Quaderno Dimostrativo
4. SEZIONE 4 - Metodi e Funzioni
5. SEZIONE 5 - Glossario
6. SEZIONE 6 - Errori Comuni
7. SEZIONE 7 - Conclusione
8. SEZIONE 8 - Checklist
9. SEZIONE 9 - Changelog

# SEZIONE 1 - Teoria

---

## 1.1 Da Bagging a Boosting: Un Cambio di Paradigma

Nella Lezione 9 abbiamo visto il Random Forest, un metodo ensemble basato sul bagging:
- Costruisce alberi indipendenti su campioni bootstrap
- Combina le predizioni per ridurre la varianza
- Gli alberi non "parlano" tra loro

Il Boosting adotta una strategia radicalmente diversa:
- Costruisce alberi sequenzialmente (uno dopo l'altro)
- Ogni nuovo albero corregge gli errori del precedente
- Si concentra sui campioni "difficili" che i modelli precedenti hanno sbagliato

---

## 1.2 L'Analogia dello Studente

| Bagging (Random Forest) | Boosting (Gradient Boosting) |
|------------------------|------------------------------|
| Studia tutto il programma piu volte | Studia, fa un test, poi studia SOLO gli argomenti sbagliati |
| Approccio "a tappeto" | Approccio "mirato agli errori" |
| Ogni sessione e indipendente | Ogni sessione dipende dalla precedente |

---

## 1.3 Il Meccanismo Sequenziale

Il Gradient Boosting costruisce il modello finale come somma di modelli deboli:

$$F_M(x) = F_0(x) + \sum_{m=1}^{M} \eta \cdot h_m(x)$$

Dove:
- $F_0(x)$ = predizione iniziale (spesso la media del target)
- $h_m(x)$ = m-esimo albero "debole" (weak learner)
- $\eta$ = learning rate (quanto "fidarsi" di ogni nuovo albero)
- $M$ = numero totale di alberi (n_estimators)

---

## 1.4 Algoritmo Passo per Passo

```
Inizializzazione:
    F0(x) = media(y)  # predizione costante iniziale

Per m = 1, 2, ..., M:
    1. Calcola i RESIDUI: ri = yi - Fm-1(xi)
       (quanto il modello attuale sbaglia per ogni campione)
    
    2. Addestra un NUOVO ALBERO hm(x) sui residui
       (l'albero impara a predire GLI ERRORI)
    
    3. Aggiorna il modello: Fm(x) = Fm-1(x) + eta * hm(x)
       (aggiungi la correzione, scalata dal learning rate)

Predizione finale: F_M(x)
```

---

## 1.5 Perche "Gradient"?

Il termine Gradient deriva dal fatto che i residui che minimizziamo corrispondono al gradiente negativo della funzione di perdita:

- Per regressione (MSE): residui = y - F(x)
- Per classificazione: si usano pseudo-residui derivati dalla log-loss

In pratica, il boosting esegue una discesa del gradiente nello spazio delle funzioni.

---

## 1.6 Random Forest vs Gradient Boosting

| Aspetto | Random Forest | Gradient Boosting |
|---------|---------------|-------------------|
| Strategia | Parallela (bagging) | Sequenziale (boosting) |
| Obiettivo | Ridurre la varianza | Ridurre il bias |
| Alberi | Profondi e indipendenti | Poco profondi e correlati |
| Errori | Mediati via voting | Corretti iterativamente |
| Velocita training | Parallelizzabile | Intrinsecamente sequenziale |
| Rischio overfitting | Basso | Piu alto (richiede tuning) |

---

## 1.7 Quando Usare Quale?

**Random Forest e preferibile quando:**
- Hai poco tempo per il tuning
- Vuoi un modello robusto out-of-the-box
- I dati hanno molto rumore
- Hai bisogno di parallelizzazione

**Gradient Boosting e preferibile quando:**
- Vuoi massimizzare le performance
- Sei disposto a fare hyperparameter tuning
- I dati hanno pattern complessi ma strutturati
- Hai risorse per evitare l'overfitting

---

## 1.8 Varianti Moderne del Gradient Boosting

| Libreria | Caratteristiche |
|----------|-----------------|
| XGBoost | Regularizzazione L1/L2, gestione missing values, parallelismo |
| LightGBM | Leaf-wise growth, molto veloce su grandi dataset |
| CatBoost | Gestione nativa delle feature categoriche |

---

## 1.9 Bias vs Variance nei Modelli ad Alberi

$$\text{Errore Totale} = \text{Bias}^2 + \text{Varianza} + \text{Rumore irriducibile}$$

| Componente | Significato | Causa |
|------------|-------------|-------|
| Bias | Errore sistematico | Modello troppo semplice |
| Varianza | Sensibilita ai dati di training | Modello troppo complesso |
| Rumore | Errore inevitabile nei dati | Intrinseco al problema |

---

## 1.10 Come gli Ensemble Riducono l'Errore

**Random Forest riduce la VARIANZA:**
- Usa alberi profondi (basso bias, alta varianza)
- La media di B alberi instabili produce predizione stabile
- Il bias rimane circa lo stesso

**Gradient Boosting riduce il BIAS:**
- Usa alberi poco profondi (alto bias, bassa varianza)
- Ogni albero corregge gli errori, bias diminuisce progressivamente
- Attenzione: troppi alberi aumentano la varianza (overfitting)

---

## 1.11 Perche il Boosting e Piu Soggetto a Overfitting

1. Correzione iterativa degli errori: ogni nuovo albero si focalizza sugli errori, incluso il rumore
2. Alberi correlati: a differenza di RF, gli alberi non sono indipendenti
3. Nessuna media: si sommano le predizioni, non si fa averaging

---

## 1.12 Segnali di Overfitting nel Boosting

| Segnale | Cosa indica |
|---------|-------------|
| Train accuracy verso 100% | Il modello memorizza i dati |
| Gap train-test che cresce | Generalizzazione che peggiora |
| Test accuracy che decresce dopo un picco | Troppi alberi |
| Learning curve che non migliora | Modello saturato |

---

## 1.13 Strategie Anti-Overfitting

1. Ridurre n_estimators: meno alberi = meno correzioni = meno rischio di fittare il rumore
2. Abbassare learning_rate: correzioni piu piccole = convergenza piu lenta ma stabile
3. Limitare max_depth: alberi piu "deboli" (depth 2-5) prevengono l'overfitting
4. Usare subsample < 1.0: campionamento casuale aggiunge regolarizzazione
5. Early Stopping: fermare il training quando la validation score smette di migliorare

---

## 1.14 La Regola d'Oro

```
n_estimators UP + learning_rate DOWN = Piu stabile, ma piu lento
learning_rate x n_estimators ~ costante
```

---

## 1.15 I Tre Parametri Fondamentali

**n_estimators - Numero di Alberi:**
| Valore | Effetto |
|--------|---------|
| Troppo basso (< 50) | Underfitting, bias alto |
| Ottimale (50-300) | Buon trade-off |
| Troppo alto (> 500) | Rischio overfitting, tempo lungo |

**learning_rate - Tasso di Apprendimento:**
| Valore | Effetto |
|--------|---------|
| Alto (0.1 - 1.0) | Convergenza veloce, rischio overfit |
| Basso (0.01 - 0.1) | Convergenza lenta, piu stabile |
| Molto basso (< 0.01) | Richiede molti alberi |

**max_depth - Profondita degli Alberi:**
| Valore | Effetto |
|--------|---------|
| 1 (decision stump) | Alberi molto deboli |
| 2-4 | Raccomandato per boosting |
| 5-8 | Per problemi complessi, rischio overfit |

---

## 1.16 Altri Parametri Utili

| Parametro | Descrizione | Valori tipici |
|-----------|-------------|---------------|
| subsample | Frazione di campioni per albero | 0.5 - 1.0 |
| min_samples_split | Campioni minimi per split | 2 - 20 |
| min_samples_leaf | Campioni minimi nelle foglie | 1 - 10 |
| max_features | Features considerate per split | sqrt, log2, 0.5 |

---

## 1.17 Configurazioni Tipiche

**Stabilita Massima:**
```python
learning_rate = 0.01
n_estimators = 1000
max_depth = 3
subsample = 0.8
```

**Velocita Massima:**
```python
learning_rate = 0.3
n_estimators = 50
max_depth = 5
subsample = 1.0
```

**Bilanciato:**
```python
learning_rate = 0.1
n_estimators = 100-200
max_depth = 3-4
subsample = 0.8
```

# SEZIONE 2 - Mappa Mentale

---

## Flusso Decisionale: Quando Usare RF vs GB

```
PROBLEMA DI CLASSIFICAZIONE/REGRESSIONE
            |
            v
    [Hai tempo per tuning?]
            |
    +-------+-------+
    |               |
   NO              SI
    |               |
    v               v
Random Forest   [Dati molto rumorosi?]
(robusto,           |
out-of-box)    +----+----+
               |         |
              SI        NO
               |         |
               v         v
          Random     Gradient
          Forest     Boosting
          (robusto)  (max performance)
```

---

## Diagnosi Overfitting nel Boosting

```
Train Accuracy ~ 100%?
        |
    +---+---+
    |       |
   NO      SI
    |       |
    v       v
   OK    [Gap train-test > 0.1?]
              |
          +---+---+
          |       |
         NO      SI
          |       |
          v       v
         OK    OVERFITTING!
                  |
                  v
          [Soluzioni]
          - Riduci n_estimators
          - Abbassa learning_rate
          - Limita max_depth
          - Usa subsample < 1.0
          - Early stopping
```

---

## Relazione tra Parametri

```
learning_rate x n_estimators ~ COSTANTE

Se learning_rate SCENDE:
    -> n_estimators deve SALIRE
    -> Training piu LENTO ma piu STABILE

Se learning_rate SALE:
    -> n_estimators puo SCENDERE
    -> Training piu VELOCE ma piu RISCHIOSO
```

---

## RF vs GB: Strategia Ensemble

```
RANDOM FOREST                    GRADIENT BOOSTING
      |                                |
      v                                v
[Albero 1] [Albero 2] ... [Albero N]   [Albero 1]
      |         |              |            |
      +---------+--------------+            v
                |                       [Albero 2] (corregge errori 1)
                v                            |
         VOTO MAGGIORANZA                    v
         (parallelo)                    [Albero 3] (corregge errori 2)
                                             |
                                             v
                                        [Albero N]
                                             |
                                             v
                                       SOMMA PESATA
                                       (sequenziale)
```

# SEZIONE 3 - Quaderno Dimostrativo

In questa sezione applichiamo i concetti appresi attraverso esercizi pratici.
Ogni esercizio include:
- Obiettivo chiaro
- Spiegazione del "perche" di ogni passaggio
- Micro-checkpoint con assert per verificare la correttezza

In [ ]:
# === ESERCIZIO 1: Visualizzazione del Boosting Sequenziale ===
# Perche: visualizziamo come il Gradient Boosting costruisce il modello
#         passo dopo passo, correggendo i residui ad ogni iterazione

import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)

# Creiamo un dataset semplice per regressione
# Perche: un dataset 1D permette di visualizzare chiaramente l'approssimazione
X_demo = np.linspace(0, 10, 100).reshape(-1, 1)
y_true = np.sin(X_demo.ravel()) + 0.5 * np.cos(2 * X_demo.ravel())
y_demo = y_true + np.random.randn(100) * 0.2

# --- MICRO-CHECKPOINT ---
assert X_demo.shape == (100, 1), "X_demo deve avere shape (100, 1)"
assert len(y_demo) == 100, "y_demo deve avere 100 elementi"
print("Micro-checkpoint 1: Dataset creato correttamente")

# Simuliamo il boosting manualmente
# Perche: capire il meccanismo interno aiuta a scegliere i parametri giusti
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Step 0: Predizione iniziale (media)
F0 = np.full_like(y_demo, y_demo.mean())
residuals = y_demo - F0

# Parametri del boosting
learning_rate = 0.5
predictions = [F0.copy()]
trees = []

for step in range(5):
    # Addestra albero sui residui
    tree = DecisionTreeRegressor(max_depth=2)
    tree.fit(X_demo, residuals)
    trees.append(tree)
    
    # Predizione dell'albero
    h = tree.predict(X_demo)
    
    # Aggiorna predizione
    F_new = predictions[-1] + learning_rate * h
    predictions.append(F_new)
    
    # Nuovi residui
    residuals = y_demo - F_new

# Visualizzazione
titles = ['Step 0: F0 = media', 'Step 1: +h1', 'Step 2: +h2', 
          'Step 3: +h3', 'Step 4: +h4', 'Step 5: +h5']

for i, ax in enumerate(axes.flat):
    ax.scatter(X_demo, y_demo, alpha=0.4, s=20, label='Dati reali', c='gray')
    ax.plot(X_demo, y_true, 'g--', linewidth=2, label='Funzione vera', alpha=0.7)
    ax.plot(X_demo, predictions[i], 'r-', linewidth=2, label=f'F_{i}(x)')
    ax.set_title(titles[i], fontsize=11)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend(loc='upper right', fontsize=8)
    ax.set_ylim([-2, 2.5])

plt.suptitle('Gradient Boosting: Correzione Sequenziale degli Errori\n(learning_rate=0.5, max_depth=2)', 
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

# --- MICRO-CHECKPOINT ---
assert len(predictions) == 6, "Devono esserci 6 predizioni (F0 a F5)"
assert len(trees) == 5, "Devono esserci 5 alberi"
print("Micro-checkpoint 2: Boosting simulato correttamente")
print("\nOSSERVAZIONE:")
print("- Ogni step aggiunge una correzione che riduce l'errore")
print("- Gli alberi 'deboli' (depth=2) contribuiscono poco singolarmente")
print("- L'ensemble finale approssima bene la funzione vera")

In [ ]:
# === ESERCIZIO 2: Confronto Bias-Variance in RF vs GB ===
# Perche: confrontiamo come RF e GB si comportano all'aumentare
#         del numero di alberi, evidenziando le differenze nel trade-off

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')

# Dataset con rumore moderato
X, y = make_classification(
    n_samples=1000, n_features=15, n_informative=8, 
    n_redundant=2, flip_y=0.1, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# --- MICRO-CHECKPOINT ---
assert X_train.shape[0] == 750, "Training set deve avere 750 campioni"
assert X_test.shape[0] == 250, "Test set deve avere 250 campioni"
print("Micro-checkpoint 1: Dataset preparato correttamente")

# Test al variare di n_estimators
n_estimators_range = [1, 5, 10, 25, 50, 100, 200]

rf_train_scores = []
rf_test_scores = []
gb_train_scores = []
gb_test_scores = []

print("Analisi Bias-Variance: RF vs GB al variare di n_estimators")
print("=" * 60)

for n_est in n_estimators_range:
    # Random Forest
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)
    rf_train_scores.append(rf.score(X_train, y_train))
    rf_test_scores.append(rf.score(X_test, y_test))
    
    # Gradient Boosting
    gb = GradientBoostingClassifier(n_estimators=n_est, max_depth=3, learning_rate=0.1, random_state=42)
    gb.fit(X_train, y_train)
    gb_train_scores.append(gb.score(X_train, y_train))
    gb_test_scores.append(gb.score(X_test, y_test))

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Random Forest
ax1 = axes[0]
ax1.plot(n_estimators_range, rf_train_scores, 'b-o', label='Train', linewidth=2, markersize=8)
ax1.plot(n_estimators_range, rf_test_scores, 'r-s', label='Test', linewidth=2, markersize=8)
ax1.fill_between(n_estimators_range, rf_train_scores, rf_test_scores, alpha=0.2, color='purple')
ax1.set_xlabel('n_estimators')
ax1.set_ylabel('Accuracy')
ax1.set_title('Random Forest: Stabilita con piu alberi', fontsize=11, fontweight='bold')
ax1.legend()
ax1.set_ylim([0.7, 1.02])
ax1.grid(True, alpha=0.3)

# Gradient Boosting
ax2 = axes[1]
ax2.plot(n_estimators_range, gb_train_scores, 'b-o', label='Train', linewidth=2, markersize=8)
ax2.plot(n_estimators_range, gb_test_scores, 'r-s', label='Test', linewidth=2, markersize=8)
ax2.fill_between(n_estimators_range, gb_train_scores, gb_test_scores, alpha=0.2, color='purple')
ax2.set_xlabel('n_estimators')
ax2.set_ylabel('Accuracy')
ax2.set_title('Gradient Boosting: Miglioramento poi rischio overfit', fontsize=11, fontweight='bold')
ax2.legend()
ax2.set_ylim([0.7, 1.02])
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# --- MICRO-CHECKPOINT ---
assert len(rf_train_scores) == len(n_estimators_range), "Devono esserci score per ogni n_estimators"
assert all(0 <= s <= 1 for s in rf_test_scores), "Gli score devono essere tra 0 e 1"
print("\nMicro-checkpoint 2: Analisi completata")
print("INTERPRETAZIONE:")
print("- RF: il gap train-test rimane STABILE con piu alberi")
print("- GB: il gap train-test puo AUMENTARE con troppi alberi")

In [ ]:
# === ESERCIZIO 3: Overfitting nel Gradient Boosting ===
# Perche: visualizziamo il fenomeno dell'overfitting nel boosting
#         su un dataset rumoroso per capire quando fermarsi

# Dataset MOLTO rumoroso per evidenziare l'overfitting
X_noisy, y_noisy = make_classification(
    n_samples=500, n_features=10, n_informative=4, 
    n_redundant=2, flip_y=0.25,  # 25% rumore!
    random_state=42
)
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X_noisy, y_noisy, test_size=0.3, random_state=42
)

# --- MICRO-CHECKPOINT ---
assert X_noisy.shape == (500, 10), "Dataset deve avere 500 campioni e 10 features"
print("Micro-checkpoint 1: Dataset rumoroso creato")

# Test con diversi n_estimators
estimators_range = range(5, 305, 10)
train_scores = []
test_scores = []

for n_est in estimators_range:
    gb = GradientBoostingClassifier(
        n_estimators=n_est, 
        max_depth=4,  # alberi abbastanza profondi
        learning_rate=0.15,  # learning rate medio-alto
        random_state=42
    )
    gb.fit(X_train_n, y_train_n)
    train_scores.append(gb.score(X_train_n, y_train_n))
    test_scores.append(gb.score(X_test_n, y_test_n))

# Trova il punto ottimale
best_idx = np.argmax(test_scores)
best_n_est = list(estimators_range)[best_idx]
best_test_score = test_scores[best_idx]

# Visualizzazione
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Curve di apprendimento
ax1 = axes[0]
ax1.plot(estimators_range, train_scores, 'b-', label='Train Accuracy', linewidth=2)
ax1.plot(estimators_range, test_scores, 'r-', label='Test Accuracy', linewidth=2)
ax1.axvline(x=best_n_est, color='green', linestyle='--', linewidth=2, label=f'Ottimo: {best_n_est}')
ax1.fill_between(estimators_range, train_scores, test_scores, alpha=0.2, color='purple', label='Gap (overfit)')
ax1.scatter([best_n_est], [best_test_score], s=100, c='green', zorder=5)
ax1.set_xlabel('n_estimators')
ax1.set_ylabel('Accuracy')
ax1.set_title('Gradient Boosting: Overfitting su Dati Rumorosi\n(flip_y=0.25, learning_rate=0.15)', fontsize=11)
ax1.legend(loc='right')
ax1.set_ylim([0.6, 1.02])
ax1.grid(True, alpha=0.3)

# Plot 2: Gap come indicatore
gaps = [tr - te for tr, te in zip(train_scores, test_scores)]
colors = ['green' if g < 0.1 else 'orange' if g < 0.2 else 'red' for g in gaps]
ax2 = axes[1]
ax2.bar(estimators_range, gaps, color=colors, width=8, alpha=0.7)
ax2.axhline(y=0.1, color='orange', linestyle='--', label='Soglia attenzione')
ax2.axhline(y=0.2, color='red', linestyle='--', label='Soglia overfitting')
ax2.set_xlabel('n_estimators')
ax2.set_ylabel('Train - Test Gap')
ax2.set_title('Gap di Generalizzazione', fontsize=11)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# --- MICRO-CHECKPOINT ---
assert best_n_est > 0, "Deve esistere un numero ottimale di alberi"
assert 0 < best_test_score < 1, "Test accuracy deve essere valida"
print(f"\nMicro-checkpoint 2: Analisi overfitting completata")
print(f"RISULTATI:")
print(f"- Numero ottimale di alberi: {best_n_est}")
print(f"- Test accuracy massima: {best_test_score:.4f}")
print(f"- Oltre {best_n_est} alberi: il modello inizia a memorizzare il rumore")

In [ ]:
# === ESERCIZIO 4: Effetto del Learning Rate ===
# Perche: il learning rate e uno dei parametri piu importanti del GB
#         e dobbiamo capire come influenza le performance

learning_rates = [0.01, 0.05, 0.1, 0.3, 0.5]
n_est_fixed = 150

results_lr = {}
for lr in learning_rates:
    gb = GradientBoostingClassifier(
        n_estimators=n_est_fixed,
        learning_rate=lr,
        max_depth=3,
        random_state=42
    )
    gb.fit(X_train, y_train)
    train_acc = gb.score(X_train, y_train)
    test_acc = gb.score(X_test, y_test)
    results_lr[lr] = {'train': train_acc, 'test': test_acc, 'gap': train_acc - test_acc}

# --- MICRO-CHECKPOINT ---
assert len(results_lr) == 5, "Devono esserci risultati per 5 learning rate"
print("Micro-checkpoint 1: Esperimenti completati")

# Visualizzazione
lrs = list(results_lr.keys())
trains = [results_lr[lr]['train'] for lr in lrs]
tests = [results_lr[lr]['test'] for lr in lrs]
gaps = [results_lr[lr]['gap'] for lr in lrs]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Train vs Test per learning rate
x = np.arange(len(lrs))
width = 0.35

ax1 = axes[0]
bars1 = ax1.bar(x - width/2, trains, width, label='Train', color='lightblue', edgecolor='navy')
bars2 = ax1.bar(x + width/2, tests, width, label='Test', color='salmon', edgecolor='darkred')
ax1.set_xlabel('learning_rate')
ax1.set_ylabel('Accuracy')
ax1.set_title(f'Effetto del Learning Rate\n(n_estimators={n_est_fixed}, max_depth=3)', fontsize=11)
ax1.set_xticks(x)
ax1.set_xticklabels([str(lr) for lr in lrs])
ax1.legend()
ax1.set_ylim([0.75, 1.02])
ax1.grid(True, alpha=0.3, axis='y')

# Valori sopra le barre
for bar, val in zip(bars2, tests):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{val:.3f}', ha='center', va='bottom', fontsize=9)

# Plot 2: Gap
colors_gap = ['green' if g < 0.05 else 'orange' if g < 0.1 else 'red' for g in gaps]
ax2 = axes[1]
ax2.bar(x, gaps, color=colors_gap, edgecolor='black')
ax2.set_xlabel('learning_rate')
ax2.set_ylabel('Train - Test Gap')
ax2.set_title('Gap di Generalizzazione per Learning Rate', fontsize=11)
ax2.set_xticks(x)
ax2.set_xticklabels([str(lr) for lr in lrs])
ax2.axhline(y=0.05, color='orange', linestyle='--', alpha=0.7)
ax2.axhline(y=0.1, color='red', linestyle='--', alpha=0.7)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# --- MICRO-CHECKPOINT ---
best_lr = max(results_lr.keys(), key=lambda x: results_lr[x]['test'])
assert 0 < best_lr <= 1, "Learning rate ottimale deve essere valido"
print(f"\nMicro-checkpoint 2: Analisi learning rate completata")
print("INTERPRETAZIONE:")
print(f"- Learning rate ottimale: {best_lr}")
print("- lr BASSO (0.01): convergenza lenta, potrebbe servire piu alberi")
print("- lr ALTO (0.3-0.5): convergenza veloce ma gap maggiore")

In [ ]:
# === ESERCIZIO 5: Confronto Completo RF vs GB ===
# Perche: eseguiamo un confronto sistematico con tutte le metriche
#         per capire quando scegliere un modello rispetto all'altro

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

# Dataset piu grande e realistico
X_full, y_full = make_classification(
    n_samples=1200, n_features=20, n_informative=12,
    n_redundant=4, n_clusters_per_class=2, flip_y=0.1, random_state=42
)

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_full, y_full, test_size=0.25, stratify=y_full, random_state=42
)

# --- MICRO-CHECKPOINT ---
assert X_train_f.shape[0] == 900, "Training set deve avere 900 campioni"
print("Micro-checkpoint 1: Dataset preparato")

# Definiamo i modelli
models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=150, max_depth=10, min_samples_split=5, random_state=42
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=150, learning_rate=0.1, max_depth=4, subsample=0.8, random_state=42
    )
}

# Confronto completo
results_comparison = {}

print("=" * 60)
print("CONFRONTO RANDOM FOREST vs GRADIENT BOOSTING")
print("=" * 60)

for name, model in models.items():
    print(f"\n{name}")
    print("-" * 50)
    
    model.fit(X_train_f, y_train_f)
    
    y_train_pred = model.predict(X_train_f)
    y_test_pred = model.predict(X_test_f)
    
    train_acc = accuracy_score(y_train_f, y_train_pred)
    test_acc = accuracy_score(y_test_f, y_test_pred)
    precision = precision_score(y_test_f, y_test_pred)
    recall = recall_score(y_test_f, y_test_pred)
    f1 = f1_score(y_test_f, y_test_pred)
    
    cv_scores = cross_val_score(model, X_full, y_full, cv=5, scoring='accuracy')
    
    results_comparison[name] = {
        'train_acc': train_acc,
        'test_acc': test_acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'gap': train_acc - test_acc
    }
    
    print(f"   Train Accuracy:    {train_acc:.4f}")
    print(f"   Test Accuracy:     {test_acc:.4f}")
    print(f"   Gap (overfit):     {train_acc - test_acc:.4f}")
    print(f"   Precision:         {precision:.4f}")
    print(f"   Recall:            {recall:.4f}")
    print(f"   F1-Score:          {f1:.4f}")
    print(f"   CV Accuracy:       {cv_scores.mean():.4f} +/- {cv_scores.std():.4f}")

# --- MICRO-CHECKPOINT ---
assert len(results_comparison) == 2, "Devono esserci risultati per entrambi i modelli"
assert all(0 < r['test_acc'] < 1 for r in results_comparison.values()), "Accuracy valide"
print("\nMicro-checkpoint 2: Confronto completato")

In [ ]:
# === ESERCIZIO 6: Feature Importance e Visualizzazione Finale ===
# Perche: confrontiamo come RF e GB assegnano importanza alle features
#         e visualizziamo tutte le metriche del confronto

# Estrai feature importance
rf_model = models['Random Forest']
gb_model = models['Gradient Boosting']

rf_importance = rf_model.feature_importances_
gb_importance = gb_model.feature_importances_

# --- MICRO-CHECKPOINT ---
assert len(rf_importance) == 20, "Devono esserci 20 feature importance"
assert np.isclose(rf_importance.sum(), 1.0), "Feature importance deve sommare a 1"
print("Micro-checkpoint 1: Feature importance estratte")

# Ordina per importanza media
avg_importance = (rf_importance + gb_importance) / 2
sorted_idx = np.argsort(avg_importance)[::-1][:10]  # Top 10

feature_names_demo = [f'Feature_{i}' for i in range(X_full.shape[1])]

# Visualizzazione completa
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Accuracy (Train, Test, CV)
model_names = list(results_comparison.keys())
ax1 = axes[0, 0]
x = np.arange(len(model_names))
width = 0.25

train_accs = [results_comparison[m]['train_acc'] for m in model_names]
test_accs = [results_comparison[m]['test_acc'] for m in model_names]
cv_means = [results_comparison[m]['cv_mean'] for m in model_names]

bars1 = ax1.bar(x - width, train_accs, width, label='Train', color='lightblue', edgecolor='navy')
bars2 = ax1.bar(x, test_accs, width, label='Test', color='salmon', edgecolor='darkred')
bars3 = ax1.bar(x + width, cv_means, width, label='CV (5-fold)', color='lightgreen', edgecolor='darkgreen')

ax1.set_ylabel('Accuracy')
ax1.set_title('Confronto Accuracy', fontsize=11, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(model_names)
ax1.legend()
ax1.set_ylim([0.75, 1.02])
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Precision, Recall, F1
ax2 = axes[0, 1]
metrics = ['precision', 'recall', 'f1']
x2 = np.arange(len(metrics))
width2 = 0.35

rf_metrics = [results_comparison['Random Forest'][m] for m in metrics]
gb_metrics = [results_comparison['Gradient Boosting'][m] for m in metrics]

bars_rf = ax2.bar(x2 - width2/2, rf_metrics, width2, label='Random Forest', color='forestgreen', alpha=0.8)
bars_gb = ax2.bar(x2 + width2/2, gb_metrics, width2, label='Gradient Boosting', color='darkred', alpha=0.8)

ax2.set_ylabel('Score')
ax2.set_title('Precision, Recall, F1', fontsize=11, fontweight='bold')
ax2.set_xticks(x2)
ax2.set_xticklabels(['Precision', 'Recall', 'F1-Score'])
ax2.legend()
ax2.set_ylim([0.75, 1.0])
ax2.grid(True, alpha=0.3, axis='y')

# Plot 3: Gap (indicatore overfitting)
ax3 = axes[1, 0]
gaps_comp = [results_comparison[m]['gap'] for m in model_names]
colors_gap = ['forestgreen', 'darkred']
bars = ax3.bar(model_names, gaps_comp, color=colors_gap, alpha=0.7, edgecolor='black')
ax3.axhline(y=0.05, color='orange', linestyle='--', label='Soglia attenzione')
ax3.axhline(y=0.1, color='red', linestyle='--', label='Soglia overfitting')
ax3.set_ylabel('Train - Test Gap')
ax3.set_title('Gap di Generalizzazione', fontsize=11, fontweight='bold')
ax3.legend(loc='upper right')
ax3.grid(True, alpha=0.3, axis='y')

for bar, g in zip(bars, gaps_comp):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, f'{g:.4f}', 
             ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 4: Feature Importance
ax4 = axes[1, 1]
x4 = np.arange(len(sorted_idx))
width4 = 0.35

bars1 = ax4.barh(x4 - width4/2, rf_importance[sorted_idx], width4, 
                label='Random Forest', color='forestgreen', alpha=0.8)
bars2 = ax4.barh(x4 + width4/2, gb_importance[sorted_idx], width4, 
                label='Gradient Boosting', color='darkred', alpha=0.8)

ax4.set_yticks(x4)
ax4.set_yticklabels([feature_names_demo[i] for i in sorted_idx])
ax4.set_xlabel('Importance')
ax4.set_title('Top 10 Feature Importance', fontsize=11, fontweight='bold')
ax4.legend(loc='lower right')
ax4.invert_yaxis()
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# --- MICRO-CHECKPOINT ---
winner_acc = max(results_comparison.items(), key=lambda x: x[1]['test_acc'])
print(f"\nMicro-checkpoint 2: Visualizzazione completata")
print(f"\nCONCLUSIONI DEL CONFRONTO:")
print(f"- Migliore Test Accuracy: {winner_acc[0]} ({winner_acc[1]['test_acc']:.4f})")
print(f"- Gap piu basso: {min(results_comparison.items(), key=lambda x: x[1]['gap'])[0]}")

# SEZIONE 4 - Metodi e Funzioni

---

## Classi Principali

| Classe | Modulo | Descrizione |
|--------|--------|-------------|
| GradientBoostingClassifier | sklearn.ensemble | Classificatore Gradient Boosting |
| GradientBoostingRegressor | sklearn.ensemble | Regressore Gradient Boosting |
| RandomForestClassifier | sklearn.ensemble | Classificatore Random Forest (confronto) |
| DecisionTreeRegressor | sklearn.tree | Albero singolo per simulazione |

---

## Parametri GradientBoostingClassifier

| Parametro | Default | Descrizione |
|-----------|---------|-------------|
| n_estimators | 100 | Numero di alberi nell'ensemble |
| learning_rate | 0.1 | Tasso di apprendimento |
| max_depth | 3 | Profondita massima degli alberi |
| min_samples_split | 2 | Campioni minimi per split |
| min_samples_leaf | 1 | Campioni minimi nelle foglie |
| subsample | 1.0 | Frazione di campioni per albero |
| max_features | None | Features considerate per split |
| random_state | None | Seed per riproducibilita |

---

## Metodi Principali

| Metodo | Descrizione |
|--------|-------------|
| fit(X, y) | Addestra il modello sui dati |
| predict(X) | Predice le classi per X |
| predict_proba(X) | Predice le probabilita per X |
| score(X, y) | Restituisce accuracy su X, y |
| feature_importances_ | Importanza delle features (attributo) |

---

## Funzioni di Valutazione

| Funzione | Modulo | Descrizione |
|----------|--------|-------------|
| cross_val_score | sklearn.model_selection | Cross-validation scores |
| train_test_split | sklearn.model_selection | Split train/test |
| accuracy_score | sklearn.metrics | Accuracy |
| precision_score | sklearn.metrics | Precision |
| recall_score | sklearn.metrics | Recall |
| f1_score | sklearn.metrics | F1-Score |

---

## Pattern di Utilizzo

```python
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split

# Split dei dati
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# Creare il modello
gb = GradientBoostingClassifier(
    n_estimators=150,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    random_state=42
)

# Addestrare
gb.fit(X_train, y_train)

# Valutare
train_score = gb.score(X_train, y_train)
test_score = gb.score(X_test, y_test)

# Cross-validation
cv_scores = cross_val_score(gb, X, y, cv=5)

# Feature importance
importance = gb.feature_importances_
```

# SEZIONE 5 - Glossario

---

| Termine | Definizione |
|---------|-------------|
| Boosting | Tecnica ensemble che costruisce modelli sequenzialmente, correggendo errori |
| Bagging | Tecnica ensemble che costruisce modelli in parallelo su campioni bootstrap |
| Gradient Boosting | Boosting che minimizza una loss function tramite discesa del gradiente |
| Weak Learner | Modello debole (es. albero poco profondo) usato nel boosting |
| Learning Rate | Parametro che controlla quanto ogni albero contribuisce alla predizione |
| n_estimators | Numero di alberi nell'ensemble |
| Residui | Differenza tra valori veri e predetti, target per il prossimo albero |
| Bias | Errore sistematico dovuto a modello troppo semplice |
| Varianza | Sensibilita del modello ai dati di training |
| Trade-off Bias-Variance | Compromesso tra underfitting (bias alto) e overfitting (varianza alta) |
| Overfitting | Modello che memorizza il training set, generalizza male |
| Underfitting | Modello troppo semplice, non cattura i pattern |
| Early Stopping | Tecnica per fermare il training quando la validation peggiora |
| Subsample | Frazione di campioni usata per addestrare ogni albero |
| max_depth | Profondita massima di ciascun albero |
| Feature Importance | Misura dell'importanza di ogni feature nel modello |
| Cross-Validation | Tecnica per stimare la performance su dati non visti |
| Gap Train-Test | Differenza tra accuracy di training e test (indicatore di overfitting) |
| XGBoost | Implementazione ottimizzata di Gradient Boosting |
| LightGBM | Gradient Boosting ottimizzato per grandi dataset |

# SEZIONE 6 - Errori Comuni

---

| Errore | Problema | Soluzione |
|--------|----------|-----------|
| learning_rate troppo alto | Overfitting rapido, instabilita | Abbassare a 0.05-0.1 |
| learning_rate troppo basso | Convergenza lenta, underfitting | Aumentare n_estimators |
| n_estimators troppo alto | Overfitting, tempo di training lungo | Usare early stopping o ridurre |
| n_estimators troppo basso | Underfitting, modello debole | Aumentare gradualmente |
| max_depth troppo alto | Alberi troppo complessi, overfitting | Limitare a 3-5 per GB |
| max_depth troppo basso | Alberi troppo deboli, underfitting | Aumentare leggermente |
| Non fare cross-validation | Stima inaffidabile delle performance | Usare sempre CV 5-fold |
| Ignorare il gap train-test | Non rilevare overfitting | Monitorare sempre il gap |
| Non normalizzare i dati | Prestazioni sub-ottimali | StandardScaler (meno critico per alberi) |
| Usare GB su dati molto rumorosi | Overfitting sul rumore | Preferire Random Forest |

# SEZIONE 7 - Conclusione

---

## Cosa Abbiamo Imparato

In questa lezione abbiamo esplorato il Gradient Boosting, un algoritmo potente che costruisce modelli sequenzialmente, correggendo gli errori dei modelli precedenti.

---

## Concetti Chiave

| Concetto | Cosa ricordare |
|----------|----------------|
| Boosting | Costruzione sequenziale, ogni modello corregge gli errori |
| RF vs GB | RF riduce varianza (parallelo), GB riduce bias (sequenziale) |
| Overfitting | GB piu soggetto, richiede tuning attento |
| Learning Rate | Piu basso = piu stabile ma piu lento |
| n_estimators | Piu alto = potenzialmente meglio, ma rischio overfit |
| max_depth | Per GB preferire alberi poco profondi (3-5) |

---

## Formula Chiave

$$F_M(x) = F_0(x) + \sum_{m=1}^{M} \eta \cdot h_m(x)$$

---

## Regola d'Oro

```
n_estimators UP + learning_rate DOWN = Piu stabile
learning_rate x n_estimators ~ costante (10-30)
```

---

## Quando Usare Cosa

| Situazione | Modello Consigliato |
|------------|---------------------|
| Dati con rumore alto | Random Forest |
| Massima performance | Gradient Boosting (con tuning) |
| Poco tempo per tuning | Random Forest |
| Dataset molto grande | RF o XGBoost/LightGBM |

---

## Prossimi Passi

Nella prossima lezione affronteremo:
- Pipeline: concatenare preprocessing + modello
- Grid Search e Random Search: ricerca automatica degli iperparametri
- Cross-Validation Stratificata: validazione robusta
- Evitare il Data Leakage

# SEZIONE 8 - Checklist

---

## Prima di Usare Gradient Boosting

- [ ] Ho compreso la differenza tra bagging e boosting
- [ ] So che GB costruisce alberi sequenzialmente
- [ ] Capisco la formula: F(x) = F0 + sum(eta * h_m(x))
- [ ] So che il learning rate controlla la velocita di apprendimento

---

## Durante l'Addestramento

- [ ] Ho scelto un learning_rate appropriato (0.05-0.2)
- [ ] Ho limitato max_depth (3-5 per GB)
- [ ] Sto monitorando il gap train-test
- [ ] Uso cross-validation per la valutazione

---

## Diagnosi Overfitting

- [ ] Il gap train-test e inferiore a 0.1
- [ ] La test accuracy non decresce con piu alberi
- [ ] Le curve di apprendimento sono stabili

---

## Confronto RF vs GB

- [ ] Ho testato entrambi i modelli
- [ ] Ho confrontato le metriche (accuracy, precision, recall, F1)
- [ ] Ho considerato il tempo di training
- [ ] Ho scelto in base al contesto (rumore, tempo, performance)

---

## Prima di Concludere

- [ ] Ho estratto le feature importance
- [ ] Ho documentato i parametri usati
- [ ] Ho salvato il modello finale

# SEZIONE 9 - Changelog

---

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-15 | Creazione iniziale del notebook |
| 2.0 | 2024-12-XX | Ristrutturazione completa secondo template a 9 sezioni |

---

## Note sulla Ristrutturazione

- Aggiunta SEZIONE 2 - Mappa Mentale con flussi decisionali
- SEZIONE 3 - Quaderno Dimostrativo con 6 esercizi pratici
- Aggiunta micro-checkpoint con assert per verifica
- Rimossi emoji secondo linee guida
- Consolidata teoria in SEZIONE 1 con 17 sottosezioni
- Aggiunta tabella metodi e funzioni
- Glossario con 20 termini chiave
- Tabella errori comuni con 10 problemi tipici